# Phase 2: Core Infrastructure & Base Framework

This notebook tests the Phase 2 components:
- Base agent framework
- Data schemas
- LLM client
- Data loaders

## Step 1: Setup and Imports

Configure Python path and import required modules.

In [ ]:
import sys
import os
from datetime import date

# Auto-detect repository path
repo_path = None
try:
    notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    if '/Repos/' in notebook_path:
        parts = notebook_path.split('/Repos/')
        if len(parts) > 1:
            repo_base = '/Workspace/Repos/' + parts[1].split('/')[0] + '/stocks-ai-system'
            if os.path.exists(repo_base):
                repo_path = repo_base
except:
    pass

if not repo_path:
    possible_paths = [
        '/Workspace/Repos/stocks-ai-system',
        '/Workspace/Users/' + os.getenv('USER', 'user') + '/stocks-ai-system',
    ]
    for path in possible_paths:
        if os.path.exists(path):
            repo_path = path
            break

if repo_path and repo_path not in sys.path:
    sys.path.insert(0, repo_path)
    print(f"✓ Added {repo_path} to Python path")
elif repo_path:
    print(f"✓ Path already configured: {repo_path}")
else:
    print("⚠ Could not auto-detect repository path. Please set repo_path manually.")

In [ ]:
# Import Phase 2 components
from src.data.schemas import (
    Company,
    FundamentalMetrics,
    ValuationMetrics,
    TechnicalIndicators,
    MacroIndicators,
    AgentPrediction
)
from src.data.loaders import YahooFinanceLoader, AlphaVantageLoader, FREDLoader
from src.agents.base_agent import BaseAgent
from src.utils.llm_client import LLMClient
from src.utils.config import config

print("✓ All imports successful")

## Step 2: Test Data Schemas

Verify that all data schemas work correctly.

In [ ]:
# Test Company schema
company = Company(
    symbol="AAPL",
    company_name="Apple Inc.",
    sector="Technology",
    industry="Consumer Electronics",
    market_cap=3000000000000
)
print(f"✓ Company schema works: {company.symbol} - {company.company_name}")

# Test AgentPrediction schema
prediction = AgentPrediction(
    prediction_id="test-123",
    symbol="AAPL",
    prediction_date=date.today(),
    target_date=date.today(),
    predicted_return=5.2,
    confidence_score=0.75,
    reasoning="Test reasoning",
    key_factors=["Factor 1", "Factor 2"]
)
print(f"✓ AgentPrediction schema works: {prediction.predicted_return}% with {prediction.confidence_score} confidence")

print("\n✅ All schemas validated")

## Step 3: Test Data Loaders

Test Yahoo Finance data loader.

In [ ]:
# Initialize data loader
yf_loader = YahooFinanceLoader()
print(f"✓ Yahoo Finance loader initialized (enabled: {yf_loader.enabled})")

# Test loading company info
symbol = "AAPL"
company_info = yf_loader.load_company_info(symbol)
if company_info:
    print(f"\n✓ Company Info Loaded:")
    print(f"  Symbol: {company_info.symbol}")
    print(f"  Name: {company_info.company_name}")
    print(f"  Sector: {company_info.sector}")
    print(f"  Industry: {company_info.industry}")
    if company_info.market_cap:
        print(f"  Market Cap: ${company_info.market_cap:,.0f}")
else:
    print(f"✗ Failed to load company info for {symbol}")

In [ ]:
# Test loading fundamentals
fundamentals = yf_loader.load_fundamentals(symbol)
if fundamentals:
    print(f"\n✓ Fundamentals Loaded:")
    print(f"  Revenue: ${fundamentals.revenue:,.0f}" if fundamentals.revenue else "  Revenue: N/A")
    print(f"  Net Income: ${fundamentals.net_income:,.0f}" if fundamentals.net_income else "  Net Income: N/A")
    print(f"  EPS: {fundamentals.eps:.2f}" if fundamentals.eps else "  EPS: N/A")
    print(f"  P/E Ratio: {fundamentals.pe_ratio:.2f}" if fundamentals.pe_ratio else "  P/E Ratio: N/A")
    print(f"  ROE: {fundamentals.roe:.2%}" if fundamentals.roe else "  ROE: N/A")
else:
    print(f"✗ Failed to load fundamentals for {symbol}")

In [ ]:
# Test loading technical indicators
technical = yf_loader.load_technical(symbol)
if technical:
    print(f"\n✓ Technical Indicators Loaded:")
    print(f"  Close Price: ${technical.close_price:.2f}" if technical.close_price else "  Close Price: N/A")
    print(f"  Volume: {technical.volume:,}" if technical.volume else "  Volume: N/A")
    print(f"  SMA 50: ${technical.sma_50:.2f}" if technical.sma_50 else "  SMA 50: N/A")
    print(f"  SMA 200: ${technical.sma_200:.2f}" if technical.sma_200 else "  SMA 200: N/A")
    print(f"  RSI: {technical.rsi:.2f}" if technical.rsi else "  RSI: N/A")
    print(f"  MACD: {technical.macd:.4f}" if technical.macd else "  MACD: N/A")
else:
    print(f"✗ Failed to load technical indicators for {symbol}")

## Step 4: Test LLM Client

Test LLM client (requires API key).

In [ ]:
# Test LLM client initialization
try:
    llm_client = LLMClient()
    print(f"✓ LLM Client initialized")
    print(f"  Provider: {llm_client.provider}")
    print(f"  Model: {llm_client.model}")
    
    # Test simple reasoning generation
    test_reasoning = llm_client.generate_reasoning(
        system_prompt="You are a helpful assistant.",
        user_prompt="In one sentence, explain what stock market prediction means.",
        temperature=0.7,
        max_tokens=100
    )
    print(f"\n✓ LLM Reasoning Test:")
    print(f"  {test_reasoning}")
    
except Exception as e:
    print(f"⚠ LLM Client test failed: {e}")
    print("  This is OK if you don't have an API key configured.")
    print("  You can still proceed with Phase 2 - agents will work once API keys are set.")

## Step 5: Test Base Agent Framework

Create a simple test agent to verify the base framework works.

In [ ]:
# Create a simple test agent
class TestAgent(BaseAgent):
    """Simple test agent for Phase 2 validation."""
    
    def __init__(self):
        super().__init__(
            name="Test Agent",
            description="A simple test agent for Phase 2 validation"
        )
        self.loader = YahooFinanceLoader()
    
    def get_system_prompt(self) -> str:
        return "You are a test stock prediction agent. Provide simple, clear predictions."
    
    def collect_signals(self, symbol: str, as_of_date: date) -> dict:
        """Collect basic signals."""
        company_info = self.loader.load_company_info(symbol)
        technical = self.loader.load_technical(symbol, as_of_date)
        
        signals = {
            "company_name": company_info.company_name if company_info else None,
            "sector": company_info.sector if company_info else None,
            "current_price": technical.close_price if technical else None,
            "sma_50": technical.sma_50 if technical else None,
        }
        return signals
    
    def analyze_signals(self, signals: dict) -> dict:
        """Simple signal analysis."""
        analysis = {
            "has_data": signals.get("current_price") is not None,
            "price_above_sma": (
                signals.get("current_price") > signals.get("sma_50")
                if signals.get("current_price") and signals.get("sma_50")
                else None
            )
        }
        return analysis

# Test agent instantiation
try:
    test_agent = TestAgent()
    print(f"✓ Test Agent created: {test_agent.name}")
    print(f"  Description: {test_agent.description}")
    
    # Test signal collection
    signals = test_agent.collect_signals("AAPL", date.today())
    print(f"\n✓ Signals collected:")
    for key, value in signals.items():
        if value is not None:
            print(f"  {key}: {value}")
    
    # Test signal analysis
    analysis = test_agent.analyze_signals(signals)
    print(f"\n✓ Analysis completed:")
    for key, value in analysis.items():
        print(f"  {key}: {value}")
    
except Exception as e:
    print(f"✗ Test Agent failed: {e}")
    import traceback
    traceback.print_exc()

## Step 6: Phase 2 Verification Summary

Verify all Phase 2 components are working.

In [ ]:
print("=" * 80)
print("PHASE 2 VERIFICATION")
print("=" * 80)

checks = {
    "Data schemas work": True,  # Tested in Step 2
    "Yahoo Finance loader works": yf_loader.enabled and company_info is not None,
    "Base agent framework works": 'test_agent' in locals(),
    "Config system works": config is not None,
}

# Check LLM client (optional)
try:
    llm_test = LLMClient()
    checks["LLM client works"] = True
except:
    checks["LLM client works"] = False

for check, status in checks.items():
    status_icon = "✓" if status else "✗"
    print(f"{status_icon} {check}")

all_critical = all([
    checks.get("Data schemas work"),
    checks.get("Yahoo Finance loader works"),
    checks.get("Base agent framework works"),
    checks.get("Config system works")
])

if all_critical:
    print("\n✅ Phase 2 Setup Complete!")
    print("You can now proceed to Phase 3: Core Agents")
    if not checks.get("LLM client works"):
        print("\n⚠️  Note: LLM client requires API key. Configure it before Phase 3.")
else:
    print("\n⚠️  Some critical checks failed. Please review the errors above.")